In [2]:
import torch.nn as nn
import torchvision.transforms as v2 
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from sklearn.metrics import accuracy_score 
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import os

In [3]:
class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, stride_depth = 1, stride_pointwise = 1):
        super().__init__()
        self.conv_depth = nn.Conv2d(in_channels = in_channels, out_channels=out_channels, kernel_size=(3,3), padding=1, stride=stride_depth, groups=in_channels)
        self.relu = nn.ReLU6(inplace=True)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.conv_pointwise = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, padding=0, stride=stride_pointwise, kernel_size=1, groups=1)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv_depth(x)))
        return self.relu(self.bn2(self.conv_pointwise(x))) 
        
class MobileNet(nn.Module):
    def __init__(self, nr_classes = 1000):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=2, padding=1)
        self.layer1 = DepthwiseSeparableConvolution(32, 64)
        self.layer2 = DepthwiseSeparableConvolution(64, 128, stride_depth=2)
        self.layer3 = DepthwiseSeparableConvolution(128, 128)
        self.layer4 = DepthwiseSeparableConvolution(128, 128, stride_depth=2)
        self.layer5 = DepthwiseSeparableConvolution(128, 256)
        self.layer6 = DepthwiseSeparableConvolution(256, 512, stride_depth=2)
        self.same_layer_sequence = nn.Sequential(*[DepthwiseSeparableConvolution(512, 512)] * 5)
        self.layer7 = DepthwiseSeparableConvolution(512, 1024, stride_depth=2)
        self.relu = nn.ReLU(inplace=True)
        self.bn32 = nn.BatchNorm2d(32)
        self.classify = nn.Sequential(*[
                nn.AdaptiveAvgPool2d(output_size=(1, 1)),
                nn.Flatten(),
                nn.Linear(1024, nr_classes),
                # nn.Softmax(dim=1),
            ])

    def forward(self, x):
        x = self.relu(self.bn32(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.same_layer_sequence(x)
        x = self.layer7(x)
        return self.classify(x)

In [4]:
PATH_MODEL = "./mobile_net.pth"
import torch.optim.sgd


class EarlyStopping:
    def __init__(self, model : MobileNet, patience, path):
        self.path = path
        self.patience = patience
        self.model = model
        self.current_patience = patience
        self.current_loss = float('inf')

    def should_continue(self, loss):
        if (loss < self.current_loss):
            self.current_loss = loss
            self.current_patience = self.patience
            torch.save(self.model.state_dict(), PATH_MODEL)
        else:
            self.current_patience -= 1
            if (self.current_patience == 0): return False
        return True

    def load_model(self):
        model = MobileNet()
        model.load_state_dict(torch.load("./mobile_net.pth", weights_only=True))
        return model
    

def initialiseTrainingModelHelpers(model, parameters):
    criterion = nn.CrossEntropyLoss()    
    # optimizer = torch.optim.Adam(model.parameters(), lr=parameters['lr_optimizer'], weight_decay=parameters['weight_decay_factor'])
    optimizer = torch.optim.SGD(model.parameters(), lr=parameters['lr_optimizer'], weight_decay=parameters['weight_decay_factor'], momentum=parameters["momentum"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                    factor=parameters['gamma_lr'], patience=parameters['patience'])
    return criterion, optimizer, lr_scheduler


def plot_losses(train_losses: list[float], validation_losses: list[float], label_x : str, label_y : str, title : str) -> None:
    """Plots training and validation losses over epochs."""
    plt.figure(figsize=(8, 6))
    plt.plot(train_losses, label=label_x, marker="o")
    plt.plot(validation_losses, label=label_y, marker="o")
    plt.xlabel(label_x)
    plt.ylabel(label_y)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

def train(model, dataloader, optimizer, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    model.to(device)
    train_loss = 0
    all_preds = []
    all_labels= []
    for idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        all_preds.extend(torch.argmax(outputs.cpu(), dim=1).detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())
    return train_loss / len(dataloader), accuracy_score(all_preds, all_labels)

def test(model, dataloader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    with torch.no_grad():
        validation_loss = 0
        all_preds = []
        all_labels= []
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            out = model(inputs)
            validation_loss += criterion(out, labels).item()
            all_preds.extend(torch.argmax(out.cpu(), dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().detach().numpy())
        return validation_loss / len(dataloader), accuracy_score(all_preds, all_labels)

def train_model(model, data, criterion, optimizer, lr_scheduler, epochs, early_stopper : EarlyStopping):
    train_info = {
        "train_loss": [],
        "validation_loss" : [],
        "train_accuracy": [],
        "validation_accuracy" : []
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    max_acc = 0
    for i in range(epochs):
        train_loss, train_acc = train(model, data["train"], optimizer, criterion)
        val_loss, val_acc = test(model, data["validation"], criterion)
        if (val_acc > max_acc):
            max_acc = val_acc
            print("acuratete maxima la epoca", i)
        lr_scheduler.step(val_loss)
        print(val_loss, val_acc, "epoch: ", i)
        train_info["train_loss"].append(train_loss)
        train_info["validation_loss"].append(val_loss)
        train_info["train_accuracy"].append(train_acc)
        train_info["validation_accuracy"].append(val_acc)
        if not early_stopper.should_continue(val_loss): break
    return train_info

def execute_pipeline(model, data, parameters):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    early_stopper = EarlyStopping(model, parameters["early_stopper"],"./model.pth")
    criterion, optimizer, lr_scheduler = initialiseTrainingModelHelpers(model, parameters)
    train_info = train_model(model, data, criterion, optimizer,
                             lr_scheduler, parameters['epochs'], early_stopper)
    plot_losses(train_info["train_loss"], train_info["validation_loss"], 
                "Epochs", "Loss", "Training and Validation Loss")
    plot_losses(train_info["train_accuracy"], train_info["validation_accuracy"], 
                "Epochs", "Accuracy", "Training and Validation Accuracy")
    new_model = early_stopper.load_model()
    _, accuracy  = test(new_model, data["validation"], criterion)
    print(accuracy)


In [4]:
class ImageNet64RawDataset(Dataset):
    def __init__(self, file_paths, transform=None):
        self.transform = transform
        self.data = []
        self.labels = []
        
        for file_path in file_paths:
            with open(file_path, 'rb') as f:
                dict = pickle.load(f)
                x = dict['data']
                y = dict['labels']
                y = [i-1 for i in y]
                self.labels.extend(y)
                num_records = len(y)
            
                for i in range(num_records):
                    image = x[i].reshape(64, 64, 3)
                    self.data.append(image)
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [8]:
import os.path
import gc
gc.collect()
torch.cuda.empty_cache()

parameters = {
    "batch_size" : 128,
    'epochs' : 200,
    'lr_optimizer': 1e-1,
    'gamma_lr': 0.3,
    'weight_decay_factor': 1e-4, # 1e-5
    'patience': 10,
    'early_stopper': 20,
    'momentum' : 0.9
}

train_transforms = Compose([
    ToTensor(),
    v2.RandomHorizontalFlip(0.5),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])

batch_files = [f'data/train_data_batch_{i}' for i in range(1, 5)]
data = {
    "train": DataLoader(ImageNet64RawDataset(file_paths=batch_files, transform=train_transforms), batch_size=parameters["batch_size"]),
    "validation": DataLoader(ImageNet64RawDataset(file_paths=["data/val_data"], transform=test_transforms), batch_size=parameters["batch_size"])
}

model = MobileNet()
if (os.path.isfile(PATH_MODEL)):
    model.load_state_dict(torch.load("./mobile_net.pth", weights_only=True))

execute_pipeline(model, data, parameters)

acuratete maxima la epoca 0
4.988056665796148 0.09942 epoch:  0
5.125753695397731 0.09108 epoch:  1
5.223733451970093 0.08296 epoch:  2
acuratete maxima la epoca 3
4.949174591952272 0.10814 epoch:  3
5.923737935702819 0.05558 epoch:  4
5.3419428457079645 0.08232 epoch:  5
5.030854592237936 0.10246 epoch:  6
4.9511966778494205 0.1077 epoch:  7
acuratete maxima la epoca 8
4.938171978191951 0.11058 epoch:  8


KeyboardInterrupt: 